In [ ]:
import numpy as np
import glob
import os 
from ultralytics import YOLO
import yolo_util as yutl
import numpy as np
import pandas as pd
import cv2
import argparse

def stream_segment(cap, start_frame, end_frame,label,raceid,FRV8,FRV8data, fr=25,windows=[32],SEQIMG_PATH="IMGSEQ_NOT_15S",FRV8PRELOAD=[],modelName ="FRV_ESDSEQ",saveinterval=15):
    print(f"Stream Segment with preload FRV8: {label}"  , start_frame-(fr*max(windows)) ,"Extract between ")
    frame_count = int(start_frame)
    print(f"SKIP TO {frame_count}")
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
    FRV8stack =[]
    frame_seq_buffer=[] 
    while cap.isOpened():
        ret, frame = cap.read()
        if frame_count > end_frame:
            break
        if ret:
            frame_count += 1
            # if frame_count <  start_frame-(fr*max(windows+1)):
            #     continue
            if frame_count % 5 == 0:
                print(frame_count)
                # frame= cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                # if len(FRV8PRELOAD) > 0:
                #     FRV8now = FRV8PRELOAD[FRV8PRELOAD["filename"]==frame_count].values
                #     # print(FRV8now)
                # else:
                #     FRV8stack, FRV8now = yutl.getDet(frameinput=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB),frame_count=frame_count,
                #                     model=FRV8,names=FRV8data,
                #                     cumulative_det=FRV8stack)
                # ---------------------------------------------------------------------------------------------------------
                frame_seq_buffer,grids = yutl.createSeqGeneral(seq_frame=frame,drawobjs=[],modelName=modelName,
                                                        list_seq_frame=frame_seq_buffer,
                                                        frame_counter=frame_count, windows=windows)
                if len(frame_seq_buffer) > 9:
                    frame_seq_buffer = frame_seq_buffer[1:]#Move Window 1+ frame and remove last
                    # SEQstack, SEQnow = yutl.getDet(frameinput=grids,frame_count=frame_count,
                    #             model=SEQ,names=SEQdata,
                    #             cumulative_det=SEQstack,batchtag=windows)
                    wx = 0
                    for grid in grids:

                        if frame_count % saveinterval == 0:
                            image_np = np.array(grid)
                            SAVEPATH =f"{SEQIMG_PATH}/SQ{windows[wx]}/{label}"
                            os.makedirs(f"{SAVEPATH}",exist_ok=True)
                            cf = int(frame_count)
                            grid.save(f"{SAVEPATH}/{raceid}_{cf}_SQ{windows[wx]}_{label}.jpg")
                            # Convert RGB to BGR
                            # image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
                            # cv2.imshow('SEQ_MONITOR', image_bgr)
                        wx = wx +1
        else:
            # Skip to the next frame if read fails
            print("Exit no more frames")
            return 0
        
print("SSSSSSSSSSSSSSSSSSSSSS")
# parser = argparse.ArgumentParser(description='Process some CSV data.')
# parser.add_argument('csv', type=str, help='The path to the CSV file',default="datapoints.csv")
# parser.add_argument('gpu', type=str, help='The gpu index' , default=0)
# parser.add_argument('raceid', type=str, help='The raceid index' , default=0)
# args = parser.parse_args()

# SEQ.tocuda(args.gpu)
# ESD.tocuda(args.gpu)
# FRV8.tocuda(args.gpu)
# print("Runngon on GPU ",args.gpu)
from concurrent.futures import ThreadPoolExecutor
def process_race(raceid, events, network_path, FRV8,FRV8data,SEQIMG_PATH="FRV_ESDSEQ",FRV8PRELOAD=[],windows=[32],modelName="FRV_ESDSEQ"):
    # try:
    onerace = events[events["raceid"] == raceid]
    print(f"{raceid} -- ",len(onerace))
    trk = str(raceid)[-5:-2]

    VID_PATH = f"{network_path}/media/amir/SSD_B/FR/FRV"
    video_path = f"{VID_PATH}/{trk}/{raceid}.mp4"
    print("Loaded Video from : ",video_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        cap.release()
        
    # try:
    print("EXTRACT list : ",len(onerace),onerace)
    for index, row in onerace.iterrows():
        start_frame = int(row['start'])
        end_frame = int(row['end'])
        label = row['label']
        print(f"Streaming segment for race {row['raceid']} ({row['label']}) from {start_frame} to {end_frame} seconds.")
        print("WOKRING ON : ",cap, start_frame, end_frame,label,raceid,SEQIMG_PATH)
        stream_segment(cap, start_frame, end_frame,label,raceid,FRV8,FRV8data, fr=25,windows=windows,SEQIMG_PATH=SEQIMG_PATH,FRV8PRELOAD=[],modelName =modelName)

    # except:

    #     pass
    if cap.isOpened():
        cap.release()
    # except:
    #     return 0

In [39]:
try:
    network_path = "/run/user/1000/gvfs/sftp:host=192.168.1.52"
    ESD_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/YOLOv8n_ESD159"
    ESD , ESDdata = yutl.loadYOLO(ESD_PATH,0)
except:
    network_path = ""
    ESD_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/YOLOv8n_ESD159"
    ESD , ESDdata = yutl.loadYOLO(ESD_PATH,0)
FRV8_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/FRV8"
FRV8 , FRV8data = yutl.loadYOLO(FRV8_PATH,0)
# SEQ_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/SEQ_X"
# SEQ , SEQdata = yutl.loadYOLO(SEQ_PATH)
video_dir = f"{network_path}/media/amir/SSD_B/FR/FRV/*/"

In [ ]:
DIRECTIONNAME = "ESDSEQ"
events = pd.read_csv(f"ESD_SEQ.csv")[["raceid", "label", "start", "end"]]
w = events[(events["start"] > 0)&(events["label"] != "NOT")]
worktodo = w[(w["label"] == "T001FRV")]
worktodo

In [ ]:

SEQIMG_PATH = f"{DIRECTIONNAME}_DATA_TEST"
events = worktodo#[:20]
with ThreadPoolExecutor(max_workers=15) as executor:
    for raceid in events["raceid"].unique():
        dst = f"{SEQIMG_PATH}/3X3/{raceid}"
        if os.path.exists(dst):
            print("Skeeep")
        else:
            # os.makedirs(dst,exist_ok=True)
            CSVnetwork_path = "/run/user/1000/gvfs/sftp:host=192.168.1.3"
            CSVnetwork_path = "NOTHING"
            preloadpath = f"{CSVnetwork_path}/media/infer/SSDC/computer_vision/extractions/FRV/FRV_OBJ_8DHORSE_000/{raceid}.csv"
            if os.path.exists(preloadpath):
                preloaddf = pd.read_csv(preloadpath)
                print("Preload Dets ",len(preloaddf) ,"Frames", int(preloaddf["filename"].max()))
                # print("Unique Frame",len(preloaddf["filename"].unique()))
                # min = int(preloaddf["filename"].min())
                # max = int(preloaddf["filename"].max())
                # print("Min Max",min,max)
                # frames = max - min
                # print("Frames",frames/5,"Unique Frames",len(preloaddf["filename"].unique()))
            else:
                preloaddf = []
            
            executor.submit(process_race, raceid, worktodo, network_path, FRV8, FRV8data, SEQIMG_PATH=SEQIMG_PATH, FRV8PRELOAD=preloaddf,windows=[3],modelName="FRV_ESDSEQ3")

In [ ]:
len(events["raceid"].unique())
events["raceid"].unique()

In [ ]:
import os 
rid = 2022052108704
if os.path.exists(f"/media/amir/SSD_B/YOLO/IMGSEQXV4/SQ16/{rid}"):
    print("Skeeep")

In [ ]:
glob.glob("/media/amir/SSD_B/YOLO/IMGSEQXV4/SQ16/2022052108704")

In [26]:
# cap = cv2.VideoCapture("/run/user/1000/gvfs/sftp:host=192.168.1.52/media/amir/SSD_B/FR/FRV/001/2023091700108.mp4")
# cap.set(cv2.CAP_PROP_POS_FRAMES, 6045)
# f= 6045
# while cap.isOpened():
#     ret, frame = cap.read()
#     if ret:
#         f = f + 1
#         if f % 5  == 0:
#             print(f,frame.shape)
#             cv2.imshow('SEQ_MONITOR', frame)
#             # opencv_image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#             # Display the image using OpenCV
#             # cv2.imshow("Image", opencv_image)
#             cv2.waitKey(1)
#     else:
#         break

# cv2.destroyAllWindows()